In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, udf
from pyspark.ml.feature import Word2Vec, Tokenizer
from pyspark.ml.linalg import Vectors, VectorUDT
import numpy as np


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1746553043698_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Start Spark session (in EMR, this is often pre-configured)
spark = SparkSession.builder.appName("UserHistoryWord2Vec").getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = spark.read.option("header", True).option("sep", "\t").csv("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/user_history.tsv")

# Show sample
df.show(5)

# Step 2: Keep only UserID and JobTitle
df = df.select("UserID", "JobTitle").dropna()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+-----+--------+--------------------+
|UserID|WindowID|Split|Sequence|            JobTitle|
+------+--------+-----+--------+--------------------+
|    47|       1|Train|       1|National Space Co...|
|    47|       1|Train|       2|   Detention Officer|
|    47|       1|Train|       3|Passenger Screene...|
|    72|       1|Train|       1|Lecturer, Departm...|
|    72|       1|Train|       2|   Student Assistant|
+------+--------+-----+--------+--------------------+
only showing top 5 rows

In [4]:
from pyspark.sql.functions import split
df = df.withColumn("JobTitleTokens", split(col("JobTitle"), "\\s+"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Step 4: Train Word2Vec on JobTitle tokens
word2vec = Word2Vec(vectorSize=100, minCount=1, inputCol="JobTitleTokens", outputCol="job_title_vector")
model = word2vec.fit(df)

# Transform to get vectors
vectorized_df = model.transform(df)

vectorized_df.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+--------------------+--------------------+
|UserID|            JobTitle|      JobTitleTokens|    job_title_vector|
+------+--------------------+--------------------+--------------------+
|    47|National Space Co...|[National, Space,...|[-0.0759868864746...|
|    47|   Detention Officer|[Detention, Officer]|[-0.3865956962108...|
|    47|Passenger Screene...|[Passenger, Scree...|[-0.0240671150386...|
|    72|Lecturer, Departm...|[Lecturer,, Depar...|[-0.0729021513834...|
|    72|   Student Assistant|[Student, Assistant]|[0.17216309159994...|
+------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [6]:
def average_vectors(vectors):
    arrs = [v.toArray() for v in vectors]
    avg = np.mean(arrs, axis=0)
    return Vectors.dense(avg)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
avg_udf = udf(average_vectors, VectorUDT())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
user_embeddings = (vectorized_df
                   .groupBy("UserID")
                   .agg(collect_list("job_title_vector").alias("vectors"))
                   .withColumn("user_job_history_vector", avg_udf("vectors"))
                   .select("UserID", "user_job_history_vector"))

user_embeddings.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------------+
| UserID|user_job_history_vector|
+-------+-----------------------+
|1000012|   [-0.0816706322237...|
|1000127|   [-0.2624434167519...|
| 100014|   [-0.1563225854188...|
|1000650|   [-0.2994719058275...|
|1000822|   [0.23477324044021...|
+-------+-----------------------+
only showing top 5 rows

In [9]:
user_embeddings.write.mode("overwrite").parquet("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/user_job_history_embeddings.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…